<a href="https://colab.research.google.com/github/DownOnCoffee/GenAI_Learning/blob/main/TextSummarizationProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
!pip install -q evaluate

In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
import evaluate
# rouge = evaluate.load("rouge")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
model_name="google/pegasus-cnn_dailymail"
InferenceModel =  AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
dataset=load_dataset("neil-code/dialogsum-test")

In [ ]:
dataset

max_length = This sets the maximum number of tokens allowed.

If the dialogue is shorter than 1024 tokens → nothing special happens

If it is longer than 1024 tokens → extra tokens are cut off

truncation = If the text is longer than max_length, truncate it instead of throwing an error

-----------

"labels" is a special, reserved key in Hugging Face Transformers.
It tells the model:
These are the correct answers the model should learn to predict

In [ ]:
def tokenize_data_func(data):
  input_encodings = tokenizer(data['dialogue'] ,max_length = 1024, truncation = True)
  output_encodings =  tokenizer(data['summary'] ,max_length = 1024, truncation = True)
  return {
      'input_ids': input_encodings['input_ids'],
      'labels': output_encodings['input_ids'],
      'attention_masks': input_encodings['attention_mask']
  }

tokenized_dataset = dataset.map(tokenize_data_func, batched= True)
tokenized_dataset

In [ ]:
tokenized_dataset['train'][8]

In [ ]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_name)

In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='pegasus-neilcode', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)